# Topic Modelling for News

![](https://images.unsplash.com/photo-1495020689067-958852a7765e?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1050&q=80)

Photo by [Roman Kraft](https://unsplash.com/photos/_Zua2hyvTBk)

This exercise is about modelling the main topics of a database of News headlines.

Begin by importing the needed libraries:

In [7]:
# TODO: import needed libraries
import nltk
import numpy as np
import pandas as pd

Load the data in the file `random_headlines.csv`

In [8]:
# TODO: load the dataset
df=pd.read_csv('random_headlines.csv')
print(df.shape)
df.head()

(20000, 2)


,publish_date,headline_text
0,20120305,ute driver hurt in intersection crash
1,20081128,6yo dies in cycling accident
2,20090325,bumper olive harvest expected
3,20100201,replica replaces northernmost sign
4,20080225,woods targets perfect season


This is always a good idea to perform some EDA (exploratory data analytics) on a dataset...

In [11]:
# TODO: Perform a short EDA
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   publish_date   20000 non-null  int64 
 1   headline_text  20000 non-null  object
dtypes: int64(1), object(1)
memory usage: 312.6+ KB


Now perform all the needed preprocessing on those headlines: case lowering, tokenization, punctuation removal, stopwords removal, stemming/lemmatization.

In [13]:
# TODO: Preprocess the input data
import string
# Tokenize
df['headline_text'] = df['headline_text'].apply(nltk.word_tokenize)
# Remove punctuation
df['headline_text']=df['headline_text'].apply(lambda tokens: [token for token in tokens if token not in string.punctuation])
# Remove stop words
df['headline_text']=df['headline_text'].apply(lambda tokens: [token for token in tokens if token.lower() not in nltk.corpus.stopwords.words('english')])
# Stem
stemmer = nltk.stem.PorterStemmer()
df['headline_text']=df['headline_text'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
df.head()

,publish_date,headline_text
0,20120305,"[ute, driver, hurt, intersect, crash]"
1,20081128,"[6yo, die, cycl, accid]"
2,20090325,"[bumper, oliv, harvest, expect]"
3,20100201,"[replica, replac, northernmost, sign]"
4,20080225,"[wood, target, perfect, season]"


Now use Gensim to compute a BOW

In [14]:
# TODO: Compute the BOW using Gensim
from gensim import corpora
dic=corpora.Dictionary(df['headline_text'])
bow=[dic.doc2bow(doc) for doc in df['headline_text']]

Compute the TF-IDF using Gensim

In [15]:
# TODO: Compute TF-IDF
from gensim.models import TfidfModel
tfidf_model=TfidfModel(bow)
tfidf=tfidf_model[bow]
print(len(tfidf))
print(tfidf[0])

20000
[(0, 0.30725466582280214), (1, 0.35289437816784547), (2, 0.42129048115131124), (3, 0.5992666854471201), (4, 0.49442279315598586)]


Finally compute the **LSA** (also called LSI) using Gensim, for a given number of Topics that you choose yourself

In [16]:
# TODO: Compute LSA
from gensim.models import LsiModel
lsi = LsiModel(tfidf, id2word=dic, num_topics=5)
print(lsi.print_topics())

/opt/conda/envs/anaconda-panel-2023.05-py310/lib/python3.11/site-packages/gensim/models/lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
/opt/conda/envs/anaconda-panel-2023.05-py310/lib/python3.11/site-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/opt/conda/envs/anaconda-panel-2023.05-py310/lib/python3.11/site-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
/opt/conda/envs/anaconda-panel-2023.05-py310/lib/python3.11/site-packages/scip

[(0, '0.457*"man" + 0.388*"polic" + 0.314*"charg" + 0.149*"court" + 0.141*"murder" + 0.127*"face" + 0.110*"crash" + 0.110*"new" + 0.109*"miss" + 0.102*"death"'), (1, '0.433*"second" + 0.408*"90" + 0.335*"abc" + 0.295*"news" + 0.287*"weather" + 0.240*"busi" + -0.236*"man" + 0.185*"sport" + -0.167*"charg" + 0.104*"plan"'), (2, '0.378*"man" + 0.273*"second" + 0.269*"charg" + 0.262*"90" + -0.212*"plan" + -0.195*"govt" + -0.194*"council" + -0.176*"new" + 0.169*"abc" + 0.159*"weather"'), (3, '-0.770*"polic" + 0.244*"man" + 0.219*"charg" + -0.163*"investig" + -0.147*"probe" + 0.140*"council" + 0.133*"plan" + 0.114*"face" + 0.113*"court" + 0.107*"govt"'), (4, '0.717*"abc" + -0.439*"second" + -0.383*"90" + 0.147*"sport" + 0.137*"market" + 0.124*"entertain" + 0.114*"busi" + 0.099*"weather" + 0.082*"analysi" + -0.074*"council"')]


For each of the topic, show the most significant words.

In [17]:
# TODO: Print the 3 or 4 most significant words of each topic
print(lsi.show_topics(num_words=4))

[(0, '0.457*"man" + 0.388*"polic" + 0.314*"charg" + 0.149*"court"'), (1, '0.433*"second" + 0.408*"90" + 0.335*"abc" + 0.295*"news"'), (2, '0.378*"man" + 0.273*"second" + 0.269*"charg" + 0.262*"90"'), (3, '-0.770*"polic" + 0.244*"man" + 0.219*"charg" + -0.163*"investig"'), (4, '0.717*"abc" + -0.439*"second" + -0.383*"90" + 0.147*"sport"')]


What do you think about those results?

Now let's try to use LDA instead of LSA using Gensim

In [18]:
# TODO: Compute LDA
from gensim.models import LdaModel
lda = LdaModel(tfidf, id2word=dic, num_topics=5)
print(lda.print_topics())

[(0, '0.004*"murder" + 0.004*"us" + 0.003*"fall" + 0.003*"court" + 0.003*"guilti" + 0.003*"man" + 0.003*"call" + 0.003*"world" + 0.003*"australia" + 0.002*"media"'), (1, '0.003*"busi" + 0.003*"rate" + 0.003*"plan" + 0.003*"new" + 0.003*"offer" + 0.002*"escap" + 0.002*"defenc" + 0.002*"sale" + 0.002*"council" + 0.002*"critic"'), (2, '0.005*"second" + 0.004*"weather" + 0.004*"polic" + 0.004*"abc" + 0.003*"assault" + 0.003*"90" + 0.003*"man" + 0.003*"review" + 0.003*"news" + 0.003*"sport"'), (3, '0.004*"polic" + 0.004*"interview" + 0.003*"alleg" + 0.003*"man" + 0.003*"arrest" + 0.003*"charg" + 0.003*"start" + 0.003*"action" + 0.003*"say" + 0.003*"hit"'), (4, '0.004*"crash" + 0.004*"polic" + 0.004*"road" + 0.003*"new" + 0.003*"urg" + 0.003*"state" + 0.003*"miss" + 0.003*"nsw" + 0.003*"probe" + 0.003*"govt"')]


In [19]:
# TODO: print the most frequent words of each topic
print(lda.show_topics(num_words=3))

[(0, '0.004*"murder" + 0.004*"us" + 0.003*"fall"'), (1, '0.003*"busi" + 0.003*"rate" + 0.003*"plan"'), (2, '0.005*"second" + 0.004*"weather" + 0.004*"polic"'), (3, '0.004*"polic" + 0.004*"interview" + 0.003*"alleg"'), (4, '0.004*"crash" + 0.004*"polic" + 0.004*"road"')]


Now, how does it work with LDA?

Let's make some visualization of the LDA results using pyLDAvis.

In [20]:
!pip install pyLDAvis --user

Looking in links: /usr/share/pip-wheels


In [21]:
# TODO: show visualization results of the LDA
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, tfidf, dic)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.035769 -0.101857       1        1  21.627088
0     -0.063514  0.167715       2        1  19.928020
2      0.160324  0.046724       3        1  19.826303
3     -0.002598 -0.061461       4        1  19.388747
1     -0.129982 -0.051121       5        1  19.229843, topic_info=         Term       Freq       Total Category  logprob  loglift
2220   second  47.000000   47.000000  Default  30.0000  30.0000
391   weather  38.000000   38.000000  Default  29.0000  29.0000
1799    alleg  29.000000   29.000000  Default  28.0000  28.0000
1868       90  32.000000   32.000000  Default  27.0000  27.0000
86    assault  34.000000   34.000000  Default  26.0000  26.0000
...       ...        ...         ...      ...      ...      ...
287       job  16.807968   38.966088   Topic5  -6.2278   0.8079
444      rise  16.763928   46.903118   Topic5  -6.2304   0.6199
186     polic  17.249363  140.600572   Topic5  -6.2018  -0.4494
606     coast  16.065031   41.128580   Topic5  -6.2730   0.7086
339    market  16.144984   56.178031   Topic5  -6.2680   0.4018

[339 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
449       4  0.937085     2014
3816      5  0.908101        3
1868      3  0.919229       90
1868      5  0.061282       90
502       2  0.291179      abc
...     ...       ...      ...
152       2  0.693005    world
152       3  0.057750    world
152       5  0.173251    world
2726      3  0.942356    worri
3600      3  0.941072  zealand

[580 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 3, 4, 2])

Depending on your results, you can try to fine tune the algorithm: number of topics, hyperparameters...
And check with others their results.